In [1]:
!pip install prf_api

  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
     |████████████████████████████████| 10.1MB 672kB/s 
     |████████████████████████████████| 102kB 580kB/s 
  Created wheel for unrar: filename=unrar-0.3-cp36-none-any.whl size=9807 sha256=fb82e0ea3fe8add9be064ae6211442ce46e060f07b7d125dfaaccf1546158dfa
  Stored in directory: /home/mayra/.cache/pip/wheels/c7/a9/1a/c5a381df313452ec2a000c9eae2188a2963df5f2f940073a5f
Successfully built unrar
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/click'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from prf_api.prf_api import PRFApi

import pandas as pd
import numpy as np

## Getting the data

In [14]:
data = PRFApi()

In [19]:
data.baixar('infracoes', anos=list(range(2015, 2019)), caminho='../data/raw')

Buscando datasets de infracoes para o ano 2015...
>> Baixando 2015/fev.csv
>> Baixando 2015/jan.csv
>> Baixando 2015/jul.csv
>> Baixando 2015/jun.csv
>> Baixando 2015/mai.csv
>> Baixando 2015/mar.csv
>> Baixando 2015/nov.csv
>> Baixando 2015/out.csv
>> Baixando 2015/set.csv
>> Baixando 2015/abr.csv
>> Baixando 2015/ago.csv
>> Baixando 2015/dez.csv
Buscando datasets de infracoes para o ano 2016...
>> Baixando 2016/jun.csv
>> Baixando 2016/mai.csv
>> Baixando 2016/mar.csv
>> Baixando 2016/nov.csv
>> Baixando 2016/out.csv
>> Baixando 2016/set.csv
>> Baixando 2016/abr.csv
>> Baixando 2016/ago.csv
>> Baixando 2016/dez.csv
>> Baixando 2016/fev.csv
>> Baixando 2016/jan.csv
>> Baixando 2016/jul.csv
Buscando datasets de infracoes para o ano 2017...
>> Baixando 2017/ago.csv
>> Baixando 2017/dez.csv
>> Baixando 2017/fev.csv
>> Baixando 2017/jan.csv
>> Baixando 2017/jul.csv
>> Baixando 2017/jun.csv
>> Baixando 2017/mai.csv
>> Baixando 2017/mar.csv
>> Baixando 2017/nov.csv
>> Baixando 2017/out.csv


In [15]:
infracoes_df = data.dataframe('infracoes', estado='RN', anos=list(range(2007, 2019)), caminho='../data/raw')

>>> Criando dataframe com ano 2007...
>>> Criando dataframe com ano 2008...


KeyboardInterrupt: 

### Analisando o dataset

Nesta seção, são feitas observações mais superficiais do dataset, buscando pontos de melhoria e inconsistências

In [6]:
infracoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361109 entries, 0 to 361108
Data columns (total 20 columns):
dat_infracao               361109 non-null object
tip_abordagem              361109 non-null object
ind_assinou_auto           120427 non-null object
ind_veiculo_estrangeiro    361109 non-null object
ind_sentido_trafego        300534 non-null object
uf_placa                   360798 non-null object
uf_infracao                361109 non-null object
num_br_infracao            361109 non-null float64
num_km_infracao            361109 non-null int64
nom_municipio              361108 non-null object
descricao_abreviada        361109 non-null object
enquadramento              361109 non-null object
data_inicio_vigencia       361109 non-null object
data_fim_vigencia          360641 non-null object
med_realizada              75179 non-null float64
med_considerada            75178 non-null float64
exc_verificado             66670 non-null float64
especie                    360405 non-n

Alguns pontos foram observados:

- O campo `dat_infracao`, `data_inicio_vigencia` e `data_fim_vigencia` devem ser campos datetime
- Campos importantes como `hora`, `uf_infracao`, `cod_infracao`,  `tip_abordagem`, entre outros, apresentam poucos dados faltantes (which is nice)
- `ind_assinou_auto`, `uf_placa`, `ind_sentido_trafego`, `especie` podem ser características relevantes , mas possuem uma porcentagem de dados faltantes
- `hora` deveria ser timestamp

In [7]:
infracoes_df.describe()

,num_br_infracao,num_km_infracao,med_realizada,med_considerada,exc_verificado
count,361109.000000,361109.000000,7.517900e+04,7.517800e+04,66670.000000
mean,233.606321,106.274095,1.249643e+03,1.217823e+03,182.157991
std,118.946852,71.723058,1.422626e+04,1.312822e+04,3372.982023
min,0.000000,0.000000,0.000000e+00,-3.000000e-02,-11000.000000
25%,101.000000,49.000000,8.900000e+01,8.200000e+01,2.000000
50%,226.000000,99.000000,9.300000e+01,8.600000e+01,6.000000
75%,304.000000,130.000000,1.010000e+02,9.400000e+01,14.000000
max,427.000000,3205.000000,3.065910e+06,3.065910e+06,815910.000000


O único campo que gerou estranheza foi o `num_br_infracao`, visto que o maior número possível de BR é 610 e a menor é 010, mas no dataset aparecem apenas aqueles entre 101 e 427.

In [8]:
infracoes_df.head()

,dat_infracao,tip_abordagem,ind_assinou_auto,ind_veiculo_estrangeiro,ind_sentido_trafego,uf_placa,uf_infracao,num_br_infracao,num_km_infracao,nom_municipio,descricao_abreviada,enquadramento,data_inicio_vigencia,data_fim_vigencia,med_realizada,med_considerada,exc_verificado,especie,nome_veiculo_marca,nom_modelo_veiculo
0,2007-09-01,S,NaN,N,C,PB,RN,304.0,230,RIACHUELO,Ultrapassar pela contramÃ£o linha de divisÃ£o ...,203 V,2000-01-01,2014-10-31,NaN,NaN,NaN,TRAÃÃO,VOLVO,NaN
1,2007-09-01,C,S,N,D,PB,RN,101.0,100,PARNAMIRIM,"Deixar de indicar c/ antec, med gesto de braÃ§...",196,2000-01-01,2016-10-31,NaN,NaN,NaN,CARGA,FORD,NaN
2,2007-09-01,S,NaN,N,C,RN,RN,226.0,69,SERRA CAIADA,"Conduzir motocicleta, motoneta e ciclomotor se...",244 I,2000-01-01,2016-02-01,NaN,NaN,NaN,PASSAGEIRO,HONDA,NaN
3,2007-09-01,S,NaN,N,C,RN,RN,226.0,69,SERRA CAIADA,"Conduzir motocicleta, motoneta e ciclomotor tr...",244 II,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,HONDA,NaN
4,2007-09-01,C,S,N,C,RN,RN,304.0,35,MOSSORO,Dirigir sob a influÃªncia de Ã¡lcool,165,2000-01-01,2012-12-20,0.94,0.86,0.57,MISTO,FORD,NaN


In [9]:
infracoes_df.query('nom_municipio == "PARELHAS" ')

,dat_infracao,tip_abordagem,ind_assinou_auto,ind_veiculo_estrangeiro,ind_sentido_trafego,uf_placa,uf_infracao,num_br_infracao,num_km_infracao,nom_municipio,descricao_abreviada,enquadramento,data_inicio_vigencia,data_fim_vigencia,med_realizada,med_considerada,exc_verificado,especie,nome_veiculo_marca,nom_modelo_veiculo
3589,2007-10-19,S,NaN,N,C,RN,RN,101.0,105,PARELHAS,Parar na pista de rolamento das rodovias,182-V,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,GM,NaN
3590,2007-10-19,S,NaN,N,C,RN,RN,101.0,105,PARELHAS,Executar operaÃ§Ã£o de retorno em locais proib...,206 I,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,VW,NaN
3591,2007-10-19,S,NaN,N,C,RN,RN,101.0,105,PARELHAS,Executar operaÃ§Ã£o de retorno em locais proib...,206 I,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,FIAT,NaN
19335,2007-03-16,C,S,N,D,RN,RN,304.0,307,PARELHAS,Conduzir veÃ­culo sem os documentos de porte o...,232,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,HONDA,NaN
22199,2007-04-16,C,S,N,D,RN,RN,101.0,105,PARELHAS,Deixar de dar passagem pela esquerda quando so...,198,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,HONDA,NaN
26229,2007-01-26,C,S,N,C,PE,RN,101.0,108,PARELHAS,Conduzir o veÃ­culo transportando passageiros ...,230 II,2000-01-01,2016-10-31,NaN,NaN,NaN,CARGA,FORD,NaN
46792,2008-05-12,C,S,N,C,PE,RN,101.0,105,PARELHAS,"NÃ£o lanÃ§ar no documento fiscal, as informaÃ§...",46 I c,2000-01-01,2012-12-31,NaN,NaN,NaN,CARGA,VW,23.220
68681,2009-09-08,C,S,N,C,RN,RN,101.0,101,PARELHAS,Conduzir o veÃ­culo c/ equip do sistema de ilu...,230 XIII,2000-01-01,2016-10-31,NaN,NaN,NaN,CARGA,TOYOTA,I/TOYOTA HILUX C
82938,2009-12-28,C,S,N,D,RN,RN,304.0,305,PARELHAS,Conduzir o veÃ­culo com qualquer uma das placa...,230 VI,2000-01-01,2016-10-31,NaN,NaN,NaN,PASSAGEIRO,GM,GM/ZAFIRA COMFOR


## Tratando os dados

O tratamento dos dados consistirá da correção dos  pontos vistos anteriormente


In [10]:
# Converte campos de data
infracoes_df.dat_infracao = pd.to_datetime(infracoes_df.dat_infracao,
                                           format='%Y-%m-%d')
infracoes_df.data_fim_vigencia = pd.to_datetime(infracoes_df.data_fim_vigencia,
                                                format='%Y-%m-%d')
infracoes_df.data_inicio_vigencia = pd.to_datetime(infracoes_df.data_inicio_vigencia,
                                                   format='%Y-%m-%d')


In [11]:
infracoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361109 entries, 0 to 361108
Data columns (total 20 columns):
dat_infracao               361109 non-null datetime64[ns]
tip_abordagem              361109 non-null object
ind_assinou_auto           120427 non-null object
ind_veiculo_estrangeiro    361109 non-null object
ind_sentido_trafego        300534 non-null object
uf_placa                   360798 non-null object
uf_infracao                361109 non-null object
num_br_infracao            361109 non-null float64
num_km_infracao            361109 non-null int64
nom_municipio              361108 non-null object
descricao_abreviada        361109 non-null object
enquadramento              361109 non-null object
data_inicio_vigencia       361109 non-null datetime64[ns]
data_fim_vigencia          360641 non-null datetime64[ns]
med_realizada              75179 non-null float64
med_considerada            75178 non-null float64
exc_verificado             66670 non-null float64
especie        

In [12]:
# Testa a conversão para datas
infracoes_df.dat_infracao.dt.weekday_name.head()

0    Saturday
1    Saturday
2    Saturday
3    Saturday
4    Saturday
Name: dat_infracao, dtype: object

Para lidar com os dados faltantes, serão removidos apenas as linhas que não possuem informação em algum dos atributos que foram tidos como os mais relevantes.

In [13]:
# Remove linhas
infracoes_df.dropna(axis=0, subset=['hora', 'ind_sentido_trafego', 'uf_placa', 
                                    'nom_modelo_veiculo'], how='any', inplace=True)

# Visualiza novo tamanho do dataset
infracoes_df.shape

KeyError: ['hora']

Outro tratamento importante é categorizar o horário do dia. Dividiremos em:
- Manhã
- Tarde
- Noite
- Madrugada

In [ ]:
def categoriza_horario(row):
  hora = row['hora']
  if hora >= '05:00' and hora < '12:00':
    return 'MANHÃ'
  elif hora >= '12:00' and hora < '17:59':
    return 'TARDE'
  elif hora >= '18:00' and hora < '23:59':
    return 'NOITE'
  else:
    return 'MADRUGADA'

infracoes_df['horario'] = infracoes_df.apply(categoriza_horario, axis=1)